In [5]:
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv('/Users/richard/Desktop/Final_Data.csv')

In [6]:
y=df['Score']
DF1=df.drop(['Name','Nationality','Score','friends','solo', 'Date_new'],axis=1)

In [16]:
df_new = df[DF1['Topic'].notnull()]
df_new.to_csv('/Users/richard/Desktop/Final_Data2.csv')

In [7]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

stemmer = SnowballStemmer("english")
def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [18]:
df_new['Topic']=df_new['Topic'].apply(remove_punctuation)
df_new['Topic']=df_new['Topic'].apply(stopwords)
df_new['Topic']=df_new['Topic'].apply(stemming)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [20]:
cv = CountVectorizer()
data = cv.fit_transform(df_new.Topic)

In [21]:
DF2=pd.concat([pd.DataFrame(data.toarray()), df_new], axis=1)

In [23]:
DF2.drop(['Name','Nationality','Score','friends','solo', 'Date_new','Topic'],axis=1,inplace=True)

In [25]:
y_new=[]
for i in y:
    if i>=8:
        a=2
    elif 6<i<8:
        a=1
    else:
        a=0
    y_new.append(a)

In [30]:
DF2=DF2.fillna('0')

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(DF2,y_new,test_size=0.1,random_state=42)

In [32]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
param_grid = {'C': np.linspace(1, 100, 1), 'penalty': ['l1', 'l2']}
grid_lr = GridSearchCV(lr, param_grid=param_grid, cv=10, n_jobs=1)

In [33]:
grid_lr.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([ 1.]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [34]:
lr=grid_lr.best_estimator_
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
y_pred_lr=lr.predict(x_test)
from sklearn.metrics import confusion_matrix
LRcm=confusion_matrix(y_test,y_pred_lr)
print(LRcm)

[[ 15   1  26]
 [ 10   6  91]
 [  5   6 401]]


In [36]:
precision=LRcm.trace()/LRcm.sum()
print(precision)

0.752228163993
